In [1]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

In [14]:
df = pd.read_csv('중학교_데이터셋.csv')
len(df)

3256

In [3]:
df.columns

Index(['학교명', '설립일자', '본교분교구분', '소재지지번주소', '소재지도로명주소', '시도교육청명', '위도', '시군구별',
       '경도', '시도명', '개방ID', '학제유형명', '2009_학생수', '2010_학생수', '2011_학생수',
       '2012_학생수', '2013_학생수', '2014_학생수', '2015_학생수', '2016_학생수', '2017_학생수',
       '2018_학생수', '2019_학생수', '2020_학생수', '2021_학생수', '2022_학생수', '2023_학생수',
       '2012_해당지역예측학생수', '2013_해당지역예측학생수', '2014_해당지역예측학생수', '2015_해당지역예측학생수',
       '2016_해당지역예측학생수', '2017_해당지역예측학생수', '2018_해당지역예측학생수', '2019_해당지역예측학생수',
       '2020_해당지역예측학생수', '2021_해당지역예측학생수', '2022_해당지역예측학생수', '2023_해당지역예측학생수',
       '2024_해당지역예측학생수', '2025_해당지역예측학생수', '2026_해당지역예측학생수', '2027_해당지역예측학생수',
       '2028_해당지역예측학생수', '2029_해당지역예측학생수', '2030_해당지역예측학생수', '2031_해당지역예측학생수',
       '2032_해당지역예측학생수', '2033_해당지역예측학생수', '2034_해당지역예측학생수', '2035_해당지역예측학생수',
       '2036_해당지역예측학생수'],
      dtype='object')

In [4]:
# X(해당지역예측학생수), Y(학생수)열 선택
columns_to_check = df.filter(regex=r'^(201[2-9]|202[0-3])_해당지역예측학생수$').columns.tolist() + \
                   df.filter(regex=r'^(201[2-9]|202[0-3])_학생수$').columns.tolist()

# 0이 포함된 행 삭제
df_nonzero = df[~(df[columns_to_check] == 0).any(axis=1)]

In [42]:
# 시군구별 출생아수와 시군구별학생수의 상관관계 분석
df_nonzero.loc[:, '상관계수'] = np.nan

for region in df_nonzero['시군구별'].unique() :
    df_temp = df[df['시군구별'] == region]
    temp_train_X = df_temp.filter(regex=r'^(201[2-9]|202[0-3])_해당지역예측학생수$').iloc[0].values
    temp_train_y = df_temp.filter(regex=r'^(201[2-9]|202[0-3])_학생수$')
    temp_train_y = temp_train_y.sum(axis=0).values
    
    # 상관계수 계산
    correlation_matrix = np.corrcoef(temp_train_X, temp_train_y)
    df_nonzero.loc[df_nonzero['시군구별'] == region, '상관계수'] = correlation_matrix[0, 1]

In [5]:
# X, Y 간 상관계수 열 추가

df_nonzero.loc[:, '상관계수'] = np.nan

for i in df_nonzero.index: 
    train_df = df_nonzero.loc[[i]]

    # X_train, y_train 선택
    X_train = train_df.filter(regex=r'^(201[2-9]|202[0-3])_해당지역예측학생수$').values.flatten()
    y_train = train_df.filter(regex=r'^(201[2-9]|202[0-3])_학생수$').values.flatten()
    
    # 상관계수 계산
    correlation_matrix = np.corrcoef(X_train, y_train)
    df_nonzero.loc[i, '상관계수'] = correlation_matrix[0, 1]

C:\Users\user\AppData\Local\Temp\ipykernel_27920\3967183013.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonzero.loc[:, '상관계수'] = np.nan


In [15]:
# 상관계수 0.8 이상인 지역 학교 filtering
df_positive = df_nonzero[df_nonzero['상관계수'] >= 0.8]
len(df_positive)

1920

In [7]:
# 2024~2036년의 "예측학생수" 열 추가 (초기값 NaN)
for year in range(2024, 2037):
    df_positive.loc[:,f'{year}_예측학생수'] = np.nan

df_positive.columns

C:\Users\user\AppData\Local\Temp\ipykernel_27920\1351284185.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positive.loc[:,f'{year}_예측학생수'] = np.nan
C:\Users\user\AppData\Local\Temp\ipykernel_27920\1351284185.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positive.loc[:,f'{year}_예측학생수'] = np.nan
C:\Users\user\AppData\Local\Temp\ipykernel_27920\1351284185.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

Index(['학교명', '설립일자', '본교분교구분', '소재지지번주소', '소재지도로명주소', '시도교육청명', '위도', '시군구별',
       '경도', '시도명', '개방ID', '학제유형명', '2009_학생수', '2010_학생수', '2011_학생수',
       '2012_학생수', '2013_학생수', '2014_학생수', '2015_학생수', '2016_학생수', '2017_학생수',
       '2018_학생수', '2019_학생수', '2020_학생수', '2021_학생수', '2022_학생수', '2023_학생수',
       '2012_해당지역예측학생수', '2013_해당지역예측학생수', '2014_해당지역예측학생수', '2015_해당지역예측학생수',
       '2016_해당지역예측학생수', '2017_해당지역예측학생수', '2018_해당지역예측학생수', '2019_해당지역예측학생수',
       '2020_해당지역예측학생수', '2021_해당지역예측학생수', '2022_해당지역예측학생수', '2023_해당지역예측학생수',
       '2024_해당지역예측학생수', '2025_해당지역예측학생수', '2026_해당지역예측학생수', '2027_해당지역예측학생수',
       '2028_해당지역예측학생수', '2029_해당지역예측학생수', '2030_해당지역예측학생수', '2031_해당지역예측학생수',
       '2032_해당지역예측학생수', '2033_해당지역예측학생수', '2034_해당지역예측학생수', '2035_해당지역예측학생수',
       '2036_해당지역예측학생수', '상관계수', '2024_예측학생수', '2025_예측학생수', '2026_예측학생수',
       '2027_예측학생수', '2028_예측학생수', '2029_예측학생수', '2030_예측학생수', '2031_예측학생수',
       '2032_예측학생수', '2033_예측학생수', '2034_예측학생수', '2035_예측학생수', '

In [8]:
# 회귀모델 적합 + 예측

for i in df_positive.index:
    train_df = df_positive.loc[[i]]
    
    # 독립변수(X), 종속변수(y) 선택
    X_train = train_df.filter(regex=r'^(201[2-9]|202[0-3])_해당지역예측학생수$').values.reshape(-1, 1)
    y_train = train_df.filter(regex=r'^(201[2-9]|202[0-3])_학생수$').values.reshape(-1, 1)
    
    # 회귀 모델 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 예측할 연도의 지역 중학생 수 데이터
    X_future = train_df.filter(regex=r'^(202[4-9]|203[0-6])_해당지역예측학생수$').values.reshape(-1, 1)
    
    # 미래 학생 수 예측
    predictions = model.predict(X_future).flatten()
    df_positive.loc[i, [f"{year}_예측학생수" for year in range(2024, 2037)]] = predictions

In [9]:
df_positive.head()

,학교명,설립일자,본교분교구분,소재지지번주소,소재지도로명주소,시도교육청명,위도,시군구별,경도,시도명,...,2027_예측학생수,2028_예측학생수,2029_예측학생수,2030_예측학생수,2031_예측학생수,2032_예측학생수,2033_예측학생수,2034_예측학생수,2035_예측학생수,2036_예측학생수
0,방이중학교,1985-04-12,본교,서울특별시 송파구 방이동 53,서울특별시 송파구 오금로11길 64,서울특별시교육청,37.515011,서울특별시 송파구,127.113244,서울,...,452.733122,407.297132,385.383732,336.871346,273.828909,222.902737,166.817686,114.850272,60.895033,21.943137
3,학산여자중학교,1951-08-06,본교,부산광역시 동래구 명장동 산85-18,부산광역시 동래구 충렬사로51번길 61,부산광역시교육청,35.204753,부산광역시 동래구,129.096168,부산,...,329.809822,296.278260,291.694809,248.875728,184.224946,130.429706,107.753686,97.983698,95.571356,70.362376
5,서울대학교사범대학부설여자중학교,1969-03-03,본교,서울특별시 종로구 동숭동 199-51,서울특별시 종로구 대학로 64,서울특별시교육청,37.577600,서울특별시 종로구,127.004295,서울,...,301.916265,274.465111,270.644589,240.080418,203.007209,178.527572,155.887445,139.473352,109.899686,91.080080
6,서산중학교,1946-12-16,본교,충청남도 서산시 석남동 57-1,충청남도 서산시 덕지천로 90,충청남도교육청,36.769046,서산시,126.458630,충남,...,866.535795,861.350438,859.010704,841.178135,820.563179,797.102600,778.764142,759.160963,749.232901,733.550358
9,중리중학교,1980-01-24,본교,대구광역시 서구 중리동 105-1,대구광역시 서구 국채보상로34길 63,대구광역시교육청,35.864715,대구광역시 서구,128.551491,대구,...,504.483432,490.718684,479.698930,463.905274,441.945330,419.189736,400.929565,388.875466,381.197442,379.606141


In [11]:
df_positive.to_csv("중학교_예측결과.csv", index=False, encoding='utf-8-sig')

In [12]:
# 2036년 기준 학생수 0명 이하인 학교 filtering
df_danger = df_positive[df_positive['2036_예측학생수'] < 0]
len(df_danger)

516

In [49]:
# df_danger.to_csv("중학교_폐교위험_지역기준.csv", index=False, encoding='utf-8-sig')

In [27]:
# 2030년 기준 학생수 0명 이하인 학교 filtering
df_danger = df_positive[df_positive['2030_예측학생수'] < 0]
#df_danger.to_csv("중학교_폐교위험_2030.csv", index=False, encoding='utf-8-sig')

In [25]:
len(df_danger)

486

In [13]:
df_danger.to_csv("중학교_폐교위험.csv", index=False, encoding='utf-8-sig')